In [5]:
import itertools
from itertools import *

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [134]:
def brute_force(lands, types):
    if len(lands) > len(types):
        return None
    
    # Try all possible combinations of assignments of types to lands
    indices = list(range(0, len(lands)))
    for index_combo in itertools.product(indices, repeat=len(types)):
        # index_combo is a tuple where each element represents the index of the land the type is
        # assigned to. e.g. (0, 0, 1) means that the first two types are assigned to land 0,
        # and the third type is assigned to land 1.
        assignment = [[] for _ in lands]
        for t, i in zip(types, index_combo):
            assignment[i].append(t)

        # Go through each land, and check:
        # - at least one type assigned to land
        # - no types assigned that it doesn't have
        # If either are false, then this assignment is invalid.
        valid = True
        for types_chosen, land in zip(assignment, lands):            
            if not types_chosen or not set(types_chosen).issubset(set(land)):
                valid = False
        if valid:
            return assignment

    return None

In [326]:
def check_algorithm(num_types):
    types = list(range(1, num_types + 1))
    print(types)
    
    for num_lands in range(1, num_types + 1):
        l = [powerset(types) for _ in range(0, num_lands)]
        for combination in itertools.product(*l):
            lands = [list(tup) for tup in combination]
            brute_force_result = brute_force(lands, types)
            algo_result = find_assignment(lands, types)
            #algo_result = brute_force_result
            print('lands: {} | brute force: {} | algorithm: {}'.format(lands, brute_force_result, algo_result))
            if (brute_force_result is None) != (algo_result is None):
                raise Exception('Differing results')


In [328]:
check_algorithm(3)

[1, 2, 3]
lands: [[]] | brute force: None | algorithm: None
lands: [[1]] | brute force: None | algorithm: None
lands: [[2]] | brute force: None | algorithm: None
lands: [[3]] | brute force: None | algorithm: None
lands: [[1, 2]] | brute force: None | algorithm: None
lands: [[1, 3]] | brute force: None | algorithm: None
lands: [[2, 3]] | brute force: None | algorithm: None
lands: [[1, 2, 3]] | brute force: [[1, 2, 3]] | algorithm: [[1, 3, 2]]
lands: [[], []] | brute force: None | algorithm: None
lands: [[], [1]] | brute force: None | algorithm: None
lands: [[], [2]] | brute force: None | algorithm: None
lands: [[], [3]] | brute force: None | algorithm: None
lands: [[], [1, 2]] | brute force: None | algorithm: None
lands: [[], [1, 3]] | brute force: None | algorithm: None
lands: [[], [2, 3]] | brute force: None | algorithm: None
lands: [[], [1, 2, 3]] | brute force: None | algorithm: None
lands: [[1], []] | brute force: None | algorithm: None
lands: [[1], [1]] | brute force: None | algor

Exception: Differing results

In [313]:
def debug(s, debug):
    if debug:
        print(s)

In [330]:
import copy

def find_assignment(lands, types, d=False):
    if len(lands) > len(types):
        return None

    if set(itertools.chain(*lands)) != set(types):
        debug(str(set(itertools.chain(*lands))), d)
        debug('not all types covered', d)
        return None
    
    lands_original = lands
    lands, types = copy.deepcopy(lands), copy.deepcopy(types)
    
    assignment = [[] for _ in lands]
    num_assigned = 0
    
    while num_assigned < len(lands):
        debug('lands: {} | types: {} | assignment: {}'.format(lands, types, assignment), d)
        
        least_flexible, min_value = None, None
        # Find type with the least # of valid lands.
        for t in types:
            num_valid_lands = len([l for l in lands if t in l])
            if nuM(min_value is None or num_valid_lands < min_value):
                min_value, least_flexible = num_valid_lands, t

        debug('least flexible: ' + str(least_flexible), d)
        if min_value == 0:
            debug('Min value == 0', d)
            return None
        
        valid_land_indices = [i for i in range(0, len(lands)) if least_flexible in lands[i]]
        debug('valid land indices: ' + str(valid_land_indices), d)
        
        candidates = []
        
        for i in valid_land_indices:
            num_options = len(lands[i])
            num_uses = len(assignment[i])
            candidates.append((i, (num_uses, num_options)))

        candidates.sort(key=lambda tup: tup[1])
        debug('candidates: ' + str(candidates), d)
        land_picked = candidates[0][0]

        assignment[land_picked].append(least_flexible)
        types.remove(least_flexible)
        lands[land_picked].clear()
        for l in lands:
            if least_flexible in l:
                l.remove(least_flexible)
        num_assigned += 1

    while types:
        t = types.pop()
        for i in range(0, len(lands_original)):
            if t in lands_original[i]:
                assignment[i].append(t)
                continue
    
    return assignment

In [331]:
result = find_assignment([[1], [1, 2, 3]], [1, 2, 3], d=True)
print(result)

lands: [[1], [1, 2, 3]] | types: [1, 2, 3] | assignment: [[], []]
least flexible: 2
valid land indices: [1]
candidates: [(1, (0, 3))]
lands: [[1], []] | types: [1, 3] | assignment: [[], [2]]
least flexible: 3
Min value == 0
None
